# ID2214 Assignment 3 Group no. [enter]
### Project members: 
[Enter Name, email]
[Enter Name, email]
[Enter Name, email]
[Enter Name, email]

### Declaration
By submitting this solution, it is hereby declared that all individuals listed above have contributed to the solution, either with code that appear in the final solution below, or with code that has been evaluated and compared to the final solution, but for some reason has been excluded. It is also declared that all project members fully understand all parts of the final solution and can explain it upon request.

It is furthermore declared that the code below is a contribution by the project members only, and specifically that no part of the solution has been copied from any other source (except for lecture slides at the course ID2214) and no part of the solution has been provided by someone not listed as project member above.

It is furthermore declared that it has been understood that no other library/package than the Python 3 standard library, NumPy, pandas and time may be used in the solution for this assignment.

### Instructions
All assignments starting with number 1 below are mandatory. Satisfactory solutions
will give 1 point (in total). If they in addition are good (all parts work more or less 
as they should), completed on time (submitted before the deadline in Canvas) and according
to the instructions, together with satisfactory solutions of assignments starting with 
number 2 below, then the assignment will receive 2 points (in total).

It is highly recommended that you do not develop the code directly within the notebook
but that you copy the comments and test cases to your regular development environment
and only when everything works as expected, that you paste your functions into this
notebook, do a final testing (all cells should succeed) and submit the whole notebook 
(a single file) in Canvas (do not forget to fill in your group number and names above).


## Load NumPy, pandas and time

In [2]:
import numpy as np
import pandas as pd
import time

## Reused functions from Assignment 1

In [3]:
# Copy and paste functions from Assignment 1 here that you need for this assignment

def create_normalization(df_init, normalizationtype = 'minmax'):
    df = df_init.copy()
    normalization = {}
    for col in df.columns:
        if(col not in ['ID', 'CLASS'] and df[col].dtype in ['float64', 'float32', 'int']):
            if(normalizationtype == 'minmax'):
                min = df[col].min()
                max = df[col].max()
                diff = max-min
                df[col] = df[col].apply(lambda x: (x-min)/(diff))
                normalization[col] = ('minmax', min, max)
                
            elif(normalizationtype == 'zscore'):
                mean = df[col].mean()
                std = df[col].std()
                df[col] = df[col].apply(lambda x: (x-mean)/(std))
                normalization[col] = ('zscore', mean, std)
              
    return(df, normalization)

def apply_normalization(df_init, normalization):
    df = df_init.copy()
    for col in df.columns:
        if(col not in ['ID', 'CLASS'] and df[col].dtype in ['float64', 'float32', 'int']):
            normalizationtype, arg1, arg2 = normalization[col]
            
            if(normalizationtype == 'minmax'):
                diff = arg2-arg1
                #min(max()) to limit the output range to [0,1], see hint 2
                df[col] = df[col].apply(lambda x: min(max((x-arg1)/(diff), 0),1))
                
            elif(normalizationtype == 'zscore'):
                df[col] = df[col].apply(lambda x: (x-arg1)/(arg2))
              
    return(df)

def create_imputation(df_init):
    df = df_init.copy()
    imputation = {}
    nrow = df.shape[0]
    for col in df.columns:
        type_col = df[col].dtype
        
        #if colonne to impute as number
        if(col not in ['ID', 'CLASS'] and type_col in ['float64', 'float32', 'int64', 'int32']):
            na = np.sum(df[col].isna())
            if(na == nrow):
                mean = 0
            else:
                mean = df[col].mean()
            df[col].fillna(mean, inplace=True)
            imputation[col] = mean
            
        #if column not to impute as object or category
        elif(col not in ["ID","CLASS"]):                        
            na = np.sum(df[col].isna())
            if(na == nrow):
                if(type_col == 'object'):
                    mode = ''
                elif(type_col == 'category'):
                    mode = df[col].cat.categories[0]
            else:
                mode = df[col].mode()[0]
            df[col].fillna(mode, inplace=True)
            imputation[col] = mode
            
    return(df, imputation)

def apply_imputation(df_init, imputation):
    df = df_init.copy()
    for col in df.columns:
        if(col not in ['ID','CLASS']):                                      
            df[col].fillna(imputation[col], inplace=True)            
    return(df)

def create_bins(df_init, nobins = 10, bintype = 'equal-width'):
    df = df_init.copy()
    binning = {}
    for col in df.columns:
        type_col = df[col].dtype
        
        if(col not in ['ID', 'CLASS'] and type_col in ['float64', 'float32', 'int64', 'int32']):
            if(bintype == 'equal-width'):
                df[col], bins = pd.cut(df[col],nobins,retbins=True, labels = False)
            else:
                # duplicate = 'drop' in case there are duplicates in the edges
                df[col], bins = pd.qcut(df[col],nobins,retbins=True, labels = False, duplicates='drop')
            bins[0], bins[-1] = -np.inf, np.inf
            binning[col] = bins
            df[col] = df[col].astype('category')
            df[col] = df[col].cat.set_categories([str(i) for i in df[col].cat.categories], rename = True)
        else:
            df[col] = df[col].astype('category')

    return(df, binning)

def apply_bins(df_init, binning):
    df = df_init.copy()
    for col in df.columns:
        type_col = df[col].dtype
        
        if(col not in ['ID', 'CLASS'] and type_col in ['float64', 'float32', 'int64', 'int32']):
            bins = binning[col]
            df[col] = pd.cut(df[col],bins,labels=False)
            df[col] = df[col].astype('category')
            df[col] = df[col].cat.set_categories([str(i) for i in df[col].cat.categories], rename = True) 
        else:
            df[col] = df[col].astype('category')

    return(df)

def split(df_init, testfraction = 0.5):
    n = df_init.shape[0]
    n_test = int(n * testfraction)
    indexes = [i for i in range(n)]
    np.random.shuffle(indexes)
    test_idx = indexes[:n_test]
    training_idx = indexes[n_test:]
    return(df_init.iloc[training_idx], df_init.iloc[test_idx])

def accuracy(predictions, correctlabels):
    correct = 0
    n = predictions.shape[0]
    for i in range(n):
        d = predictions.iloc[i]
        pred  = d.idxmax()
        if(pred == correctlabels[i]):
            correct += 1
    return(correct/n)

def create_one_hot(df_init):
    df = df_init.copy()
    df2 = df.copy()
    one_hot = {}
    for col in df.columns:
        if((hasattr(df[col], 'object') or hasattr(df[col], 'category')) and col not in ['ID','CLASS']): 
            df[col] = df[col].astype('category')
            n_cat = len(df[col].cat.categories)
            one_hot[col] = df[col].cat.categories
            for i in one_hot[col]:
                name = col+'_'+i   
                new_col = df[col]==i
                new_col = new_col.astype('float')
                df2[name]=new_col 
            df2 = df2.drop(columns = col, axis = 1) 
    return(df2, one_hot)

def apply_one_hot(df_init, one_hot):
    df = df_init.copy()
    df2 = df.copy()
    for col in df.columns:
        if(col in one_hot.keys()):
            for i in one_hot[col]:
                name = col+'-'+i
                new_col = df[col]==i
                new_col = pd.Series(new_col.astype('float'))
                df2[name] = new_col
            df2 = df2.drop(columns = col, axis = 1)
            
    return(df2)

def folds(df, nofolds):
    shuffling = np.random.permutation(df.index)
    intervals = [int(i*df.shape[0]/nofolds) for i in range(nofolds+1)]
    listDf = [df.iloc[intervals[i]:intervals[i+1],:] for i in range(nofolds)]
    return(listDf)

def brier_score(df, correctlabels):
    n = df.shape[0]
    avg_error = 0
    for i in range(n):
        v = np.array(df.iloc[i,:])
        idx = np.where(df.columns==correctlabels[i])[0]
        correct = [1 if i == idx else 0 for i in range(df.shape[1])]
        error = np.sum((v-correct)**2)
        avg_error += error
    avg_error = avg_error / n
    return(avg_error)

def get_true_false_positive(prediction_vector, label_vector, label):
    pred = np.array(prediction_vector)
    v = [i == label for i in label_vector] 
    this_label, not_this_label = pred[v], pred[[not i for i in v]]
    return(this_label, not_this_label)

def get_list(prediction_vector, label_vector, label):
    this_label, not_this_label = get_true_false_positive(prediction_vector, label_vector, label)
    scores_false = [0]
    scores_false += sorted(prediction_vector)
    scores_false += [1]
    dict_scores = {}
    for i in scores_false:
        if(len(not_this_label)==0):
            false_pos_i_r = 0
        else:
            false_pos_i_r = np.sum(not_this_label>=i)/len(not_this_label)
        if(len(this_label)==0):
            true_pos_i_r = 0
        else:
            true_pos_i_r = np.sum(this_label>=i)/len(this_label)
        dict_scores[i] = [false_pos_i_r, true_pos_i_r]
    list_reversed = [i for i in reversed(list(dict_scores.values()))]
    return(list_reversed)

def get_area(list_values):
    area = 0
    if([0,0] not in list_values):
        list_values = [[0,0]]+list_values
    if([1,1] not in list_values):
        list_values = list_values+[[1,1]]
    n = len(list_values)
    for i in range(n-1):
        left, right = i, i+1
        tpr_left, tpr_right = list_values[left][1], list_values[right][1]
        fpr_left, fpr_right = list_values[left][0], list_values[right][0]
        if(fpr_right==fpr_left):
            next
        height = (tpr_left+tpr_right)/2
        width = fpr_right-fpr_left
        area += height*width
    return(area)

def get_frequencies(correctlabels, nClasses):
    x = pd.Series(correctlabels)
    frequencies = x.value_counts(normalize = True)
    d_freq = dict(frequencies)
    freqs = []
    for i in range(1,nClasses+1):
        if(i not in d_freq):
            d_freq[i] = 0
    return(d_freq)
    
def auc(df, correctlabels):
    nClasses = df.shape[1]
    frequencies = get_frequencies(correctlabels, nClasses)

    area = 0
    for col in df.columns:
        prediction_vector = df[col]
        l = get_list(prediction_vector, correctlabels, col)
        area_col = get_area(l)
        area += frequencies[col]*area_col
    return(area)

## 1. Define the class DecisionTree

In [198]:
import cProfile
from datetime import datetime

In [746]:
class DecisionTree:
    
    def __init__(self):
        self.binning = None
        self.imputation = None
        self.labels = None
        self.model = None
        # nodenomax is the maximum nodeno which has been given to a tree so it is a reference to know what should
        # be the nodeno of the next node
        self.nodenomax = 0
        
    ## This function takes a list of feature and a dataframe and gives the feature_to_select: the feature which minimises 
    ## the residual information and the list of sub_features which is the features minus the feature_to_select
    def select_feature_min_inf(self,features, df):
        # spec_agg is a special aggregation function
        # the input df is a groupedby dataframe, which is a groupedby df by featureI (one feature selected) of a dataframe
        # which is itself a size() dataframe by (featureI, class)
        # This aim of this function is to give the residual information for one feature
        # NB: We do not divide by the total cardinal as it would be the same for all features.
        def spec_agg(df):
            return(-np.sum(df*np.log(df/np.sum(df))))
    
        idx, minInf = 0, 1e5
        for i in range(len(features)):
            f = features[i]
            # observed = True allows to remove the NA version of counts for categorical data, sort False for speed
            class_repart = df.groupby([f, 'CLASS'], observed=True, sort = False).size()
            # By aggregating with spec_agg we compute the inf residual in an efficient way
            inf = class_repart.groupby([f], sort = False).agg(spec_agg).sum()
            if(inf < minInf):
                minInf = inf
                idx = i
        feature_to_select = features[idx]
        sub_features = features[:idx] + features[idx+1:]
        return(feature_to_select, sub_features)
    
    
    def divide_and_conquer(self, df, features, class_freq, min_samples_split, nodeno):
        feature_to_select, sub_features = self.select_feature_min_inf(features,df)
        #Generating the sub trees
        values = df[feature_to_select].unique()
        trees = []
        sub_nodeno = self.nodenomax
        nodedict = {}
        # Made to improve the speed
        df.set_index(feature_to_select, inplace=True)
        for i in values:
            sub_df = df.loc[[i]]
            sub_class_freq = sub_df.CLASS.value_counts(normalize=True)
            self.nodenomax += 1
            nodedict[i] = self.nodenomax
            # We are checking if we should create a new sub tree or if we should create a leaf
            # leaf if we have less instances than min_sample or if we don't have anymore features
            if((sub_df.shape[0]<=min_samples_split or len(sub_features)==0)):
                sub_tree = ((self.nodenomax, "leaf", sub_class_freq))
            else:
                sub_tree = self.divide_and_conquer(sub_df, sub_features, sub_class_freq, min_samples_split, self.nodenomax)
                
            self.model.append(sub_tree)
        this_tree = (nodeno, feature_to_select, nodedict)
        return(this_tree)
    
    def fit(self, df, nobins=10, bintype='equal-width', min_samples_split = 5):
        self.__init__()
        self.model = []
        df, self.imputation = create_imputation(df)
        classColumn = df.loc[:,'CLASS'].astype('category')
        self.labels = classColumn.cat.categories
        df, self.binning = create_bins(df, nobins = nobins, bintype = bintype)
        features = df.columns
        features = list(features.drop(['CLASS', 'ID']))
        class_freq = df.CLASS.value_counts(normalize=True).sort_index()
        root_node = self.divide_and_conquer(df, features, class_freq, min_samples_split, 0)
        self.model.append(root_node)
        # We sort given the nodeno
        self.model = sorted(self.model, key=lambda x:x[0])
        
    ## This recursive funtion takes a row and a nodeno and will go through the nodes to find the probabilities
    ## giving back a vector of size number of labels
    def make_prediction(self, nodeno, row):
        nodeno, f, d = self.model[nodeno]
        if(f == "leaf"):
            return(d.sort_index())
        else:
            value = row[f]
            # We have some case where we don't have any node for the values of row
            if(value not in d.keys()):
                # if so we will go in the deepest node compatible and average the predictions of the sub_nodes
                v = list(d.values())
                m = np.zeros([len(v), len(self.labels)])
                for i in range(len(v)):
                     m[i] = self.make_prediction(v[i], row)
                m = np.mean(m, axis = 0)
                return(m)
            # else we just iterate in the good next node
            else:
                new_nodeno = d[value]
            return(self.make_prediction(new_nodeno, row))
        
    def predict(self, df):
        df = df.drop(['ID', 'CLASS'], axis = 1)
        df = apply_imputation(df_init=df, imputation=self.imputation)
        df = apply_bins(df_init=df, binning=self.binning)
        matrix = np.array([np.zeros(len(self.labels)) for i in range (df.shape[0])])
        for i in range(df.shape[0]):
            row = df.iloc[i]
            matrix[i] = self.make_prediction(0, row)
        probs = pd.DataFrame(matrix, columns=self.labels)
        # We need to add the predictions of the labels that might not have been present in the training set
        # for example label 4 is never represented so it should appear wth a prob of 0 for all rows
        max_i = np.max(a=np.array(probs.columns, dtype='int'))
        for i in range(1,max_i):
            if (i not in probs.columns):
                probs[i] = 0
        cols = sorted(predictions.columns)
        probs = probs.loc[:,cols]
        return(probs)

In [747]:
# Test your code (leave this part unchanged, except for if auc is undefined)

glass_train_df = pd.read_csv("glass_train.txt")

glass_test_df = pd.read_csv("glass_test.txt")

tree_model = DecisionTree()

test_labels = glass_test_df["CLASS"]

nobins_values = [5,10]
bintype_values = ["equal-width","equal-size"]
min_samples_split_values = [3,5,10]
parameters = [(nobins,bintype,min_samples_split) for nobins in nobins_values for bintype in bintype_values 
              for min_samples_split in min_samples_split_values]

results = np.empty((len(parameters),3))

for i in range(len(parameters)):
    t0 = time.perf_counter()
    tree_model.fit(glass_train_df,nobins=parameters[i][0],bintype=parameters[i][1],min_samples_split=parameters[i][2])
    print("Training time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    t0 = time.perf_counter()
    predictions = tree_model.predict(glass_test_df)
    print("Testing time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    results[i] = [accuracy(predictions,test_labels),brier_score(predictions,test_labels),
                  auc(predictions,test_labels)] # Assuming that you have defined auc - remove otherwise

results = pd.DataFrame(results,index=pd.MultiIndex.from_product([nobins_values,bintype_values,min_samples_split_values]),
                       columns=["Accuracy","Brier score","AUC"])

results

Training time (5, 'equal-width', 3): 3.60 s.
Testing time (5, 'equal-width', 3): 0.26 s.
Training time (5, 'equal-width', 5): 1.84 s.
Testing time (5, 'equal-width', 5): 0.33 s.
Training time (5, 'equal-width', 10): 0.91 s.
Testing time (5, 'equal-width', 10): 0.52 s.
Training time (5, 'equal-size', 3): 2.97 s.
Testing time (5, 'equal-size', 3): 0.16 s.
Training time (5, 'equal-size', 5): 2.94 s.
Testing time (5, 'equal-size', 5): 0.63 s.
Training time (5, 'equal-size', 10): 2.02 s.
Testing time (5, 'equal-size', 10): 0.26 s.
Training time (10, 'equal-width', 3): 2.49 s.
Testing time (10, 'equal-width', 3): 0.33 s.
Training time (10, 'equal-width', 5): 2.19 s.
Testing time (10, 'equal-width', 5): 0.28 s.
Training time (10, 'equal-width', 10): 0.95 s.
Testing time (10, 'equal-width', 10): 0.14 s.
Training time (10, 'equal-size', 3): 1.85 s.
Testing time (10, 'equal-size', 3): 0.23 s.
Training time (10, 'equal-size', 5): 1.45 s.
Testing time (10, 'equal-size', 5): 0.21 s.
Training time (

Accuracy  Brier score       AUC
5  equal-width 3   0.635514     0.569085  0.787104
               5   0.607477     0.573103  0.789411
               10  0.570093     0.565079  0.791692
   equal-size  3   0.598131     0.689916  0.749279
               5   0.635514     0.630872  0.762795
               10  0.598131     0.652672  0.743455
10 equal-width 3   0.654206     0.535986  0.833842
               5   0.598131     0.544863  0.824130
               10  0.588785     0.509364  0.839930
   equal-size  3   0.485981     0.907885  0.653384
               5   0.485981     0.907745  0.652530
               10  0.504673     0.823100  0.649869

In [748]:
tree_model = DecisionTree()
train_labels = glass_train_df["CLASS"]
tree_model.fit(glass_train_df,min_samples_split=1)
predictions = tree_model.predict(glass_train_df)
print("Accuracy on training set: {0:.2f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.2f}".format(auc(predictions,train_labels)))
print("Brier score on training set: {0:.2f}".format(brier_score(predictions,train_labels)))

Accuracy on training set: 0.97
AUC on training set: 1.00
Brier score on training set: 0.03


In [ ]:
# Define the class DecisionTree with three functions __init__, fit and predict (after the comments):
#
# Input to __init__: 
# self: the object itself
#
# Output from __init__:
# nothing
# 
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# binning, imputatiom, labels, model
#
# Input to fit:
# self: the object itself
# df: a dataframe (where the column names "CLASS" and "ID" have special meaning)
# nobins: no. of bins (default = 10)
# bintype: either "equal-width" (default) or "equal-size"
# min_samples_split: no. of instances required to allow a split (default = 5)
#
# Output from fit:
# nothing
#
# The result of applying this function should be:
#
# self.binning should be a discretization mapping (see Assignment 1) from df
# self.imputation should be an imputation mapping (see Assignment 1) from df
# self.labels should be the categories of the "CLASS" column of df, set to be of type "category" 
# self.model should be a decision tree (for details, see lecture slides), where the leafs return class probabilities
# Note that the function does not return anything but just assigns values to the attributes of the object.
#
# Hint 1: First find the available features (excluding "CLASS" and "ID"), then find the class counts, e.g., using 
#         groupby, and calculate the default class probabilities (relative frequencies of the class labels)
# Hint 2: Define a function, e.g., called divide_and_conquer, that takes the above as input together with df 
#         and min_samples_split, and also a nodeno (starting with 0) to keep track of the generated nodes in the tree
# Hint 3: You may represent the tree under construction as a list of nodes (tuples), on the form:
#         (nodeno,"leaf",class_probabilities): corresponding to a leaf node where class_probabilities is a vector
#                                              with the relative class frequencies (ordered according to self.labels)
#         (nodeno,feature,node_dict): corresponding to an internal (non-leaf) node where node_dict is a mapping from
#                                     the possible values of feature to child nodes (their nodenos)
# Hint 4: You may evaluate each feature by a function information_content, which takes the group sizes
#         for each possible value of the feature together with the class counts of each group as input
# Hint 5: The best feature found (with lowest resulting information content) will be used to split the training
#         instances, and each sub-group is used for generating a sub-tree (recursively by divide_and_conquer,
#         see lecture slides for details)
# Hint 6: The list of nodes output by divide_and_conquer may finally be converted to an array, where each nodeno in the 
#         tuples corresponds to an index of the array 
#
# Input to predict:
# self: the object itself
# df: a dataframe
# 
# Output from predict:
# predictions: a dataframe with class labels as column names and the rows corresponding to
#              predictions with estimated class probabilities for each row in df, where the class probabilities
#              are the relative class frequencies in the leaves of the decision tree into which the instances in
#              df fall
#
# Hint 1: Drop any "CLASS" and "ID" columns first and then apply imputation and binning
# Hint 2: Iterate over the rows calling some sub-function, e.g., make_prediction(nodeno,row), which for a test row
#         finds a leaf node from which class probabilities are obtained
# Hint 3: This sub-function may recursively traverse the tree (represented by an array), starting with the nodeno
#         that corresponds to the root



## Instructions results

In [14]:
# Test your code (leave this part unchanged, except for if auc is undefined)

glass_train_df = pd.read_csv("glass_train.txt")

glass_test_df = pd.read_csv("glass_test.txt")

tree_model = DecisionTree()

test_labels = glass_test_df["CLASS"]

nobins_values = [5,10]
bintype_values = ["equal-width","equal-size"]
min_samples_split_values = [3,5,10]
parameters = [(nobins,bintype,min_samples_split) for nobins in nobins_values for bintype in bintype_values 
              for min_samples_split in min_samples_split_values]

results = np.empty((len(parameters),3))

for i in range(len(parameters)):
    t0 = time.perf_counter()
    tree_model.fit(glass_train_df,nobins=parameters[i][0],bintype=parameters[i][1],min_samples_split=parameters[i][2])
    print("Training time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    t0 = time.perf_counter()
    predictions = tree_model.predict(glass_test_df)
    print("Testing time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    results[i] = [accuracy(predictions,test_labels),brier_score(predictions,test_labels),
                  auc(predictions,test_labels)] # Assuming that you have defined auc - remove otherwise

results = pd.DataFrame(results,index=pd.MultiIndex.from_product([nobins_values,bintype_values,min_samples_split_values]),
                       columns=["Accuracy","Brier score","AUC"])

results



Training time (5, 'equal-width', 3): 1.58 s.
Testing time (5, 'equal-width', 3): 0.11 s.
Training time (5, 'equal-width', 5): 1.33 s.
Testing time (5, 'equal-width', 5): 0.11 s.
Training time (5, 'equal-width', 10): 0.74 s.
Testing time (5, 'equal-width', 10): 0.11 s.
Training time (5, 'equal-size', 3): 1.60 s.
Testing time (5, 'equal-size', 3): 0.07 s.
Training time (5, 'equal-size', 5): 0.84 s.
Testing time (5, 'equal-size', 5): 0.07 s.
Training time (5, 'equal-size', 10): 0.44 s.
Testing time (5, 'equal-size', 10): 0.07 s.
Training time (10, 'equal-width', 3): 2.47 s.
Testing time (10, 'equal-width', 3): 0.06 s.
Training time (10, 'equal-width', 5): 1.56 s.
Testing time (10, 'equal-width', 5): 0.06 s.
Training time (10, 'equal-width', 10): 0.88 s.
Testing time (10, 'equal-width', 10): 0.06 s.
Training time (10, 'equal-size', 3): 1.51 s.
Testing time (10, 'equal-size', 3): 0.07 s.
Training time (10, 'equal-size', 5): 1.32 s.
Testing time (10, 'equal-size', 5): 0.07 s.
Training time (

Accuracy  Brier score       AUC
5  equal-width 3   0.635514     0.566861  0.794838
               5   0.644860     0.530257  0.812039
               10  0.579439     0.540913  0.802684
   equal-size  3   0.644860     0.678333  0.757026
               5   0.635514     0.644065  0.759834
               10  0.598131     0.652672  0.743455
10 equal-width 3   0.654206     0.566439  0.816216
               5   0.626168     0.529316  0.832679
               10  0.598131     0.503810  0.845034
   equal-size  3   0.504673     0.905079  0.656123
               5   0.504673     0.902743  0.655598
               10  0.532710     0.868821  0.669287

In [15]:
train_labels = glass_train_df["CLASS"]
tree_model.fit(glass_train_df,min_samples_split=1)
predictions = tree_model.predict(glass_train_df)
print("Accuracy on training set: {0:.2f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.2f}".format(auc(predictions,train_labels)))
print("Brier score on training set: {0:.2f}".format(brier_score(predictions,train_labels)))

Accuracy on training set: 0.97
AUC on training set: 1.00
Brier score on training set: 0.03


### Comment on assumptions, things that do not work properly, etc.


## 2. Define the class DecisionForest

In [749]:
class DecisionForest:
    
    def __init__(self):
        self.binning = None
        self.imputation = None
        self.labels = None
        # models is now a list of a list. So it is the list of trees, i.e. the list of the list of nodes
        self.models = None
        # nodenomax is the maximum nodeno which has been given to a tree so it is a reference to know what should
        # be the nodeno of the next node
        # nodenomax is now  list of the nodenomax of the trees
        self.nodenomax = None
        self.notrees = None
        
    ## This function takes a list of feature and a dataframe and gives the feature_to_select: the feature which minimises 
    ## the residual information and the list of sub_features which is the features minus the feature_to_select
    def select_feature_min_inf(self,features, df):
        # spec_agg is a special aggregation function
        # the input df is a groupedby dataframe, which is a groupedby df by featureI (one feature selected) of a dataframe
        # which is itself a size() dataframe by (featureI, class)
        # This aim of this function is to give the residual information for one feature
        # NB: We do not divide by the total cardinal as it would be the same for all features.
        def spec_agg(df):
            return(-np.sum(df*np.log(df/np.sum(df))))
    
        idx, minInf = 0, 1e5
        for i in range(len(features)):
            f = features[i]
            # observed = True allows to remove the NA version of counts for categorical data, sort False for speed
            class_repart = df.groupby([f, 'CLASS'], observed=True, sort = False).size()
            # By aggregating with spec_agg we compute the inf residual in an efficient way
            inf = class_repart.groupby([f]).agg(spec_agg).sum()
            if(inf < minInf):
                minInf = inf
                idx = i
        feature_to_select = features[idx]
        if(idx==0):
            sub_features = list(features[1:])
        elif(idx==len(features)):
            sub_features = list(features[:-1])
        else:
            sub_features = list(features[:idx]) + list(features[idx+1:])
        return(feature_to_select, sub_features)
    
    def divide_and_conquer(self, df, features, class_freq, min_samples_split, nodeno, notree):
        feature_to_select, sub_features = self.select_feature_min_inf(features,df)
        #Generating the sub trees
        values = df[feature_to_select].unique()
        trees = []
        sub_nodeno = self.nodenomax[notree]
        nodedict = {}
        # Made to improve the speed
        df.set_index(feature_to_select, inplace=True)
        for i in values:
            sub_df = df.loc[[i]]
            sub_class_freq = sub_df.CLASS.value_counts(normalize=True)
            self.nodenomax[notree] += 1
            nodedict[i] = self.nodenomax[notree]
            # We are checking if we should create a new sub tree or if we should create a leaf
            # leaf if we have less instances than min_sample or if we don't have anymore features
            if((sub_df.shape[0]<=min_samples_split or len(sub_features)==0)):
                sub_tree = ((self.nodenomax[notree], "leaf", sub_class_freq))
            else:
                sub_tree = self.divide_and_conquer(sub_df, sub_features, sub_class_freq, min_samples_split, self.nodenomax[notree], notree)
                
            self.models[notree].append(sub_tree)
        this_tree = (nodeno, feature_to_select, nodedict)
        return(this_tree)
        
    def fit(self, df, nobins = 10, bintype = 'equal-width', min_samples_split = 5, random_features = 2, notrees = 10):
        self.__init__()
        if(random_features==0):
            random_features = df.shape[1]-2
        self.notrees = notrees
        df, self.imputation = create_imputation(df)
        classColumn = df.loc[:,'CLASS'].astype('category')
        self.labels = classColumn.cat.categories
        df, self.binning = create_bins(df, nobins = nobins, bintype = bintype)
        self.nodenomax = np.zeros(notrees, dtype='int')
        self.models = [[] for i in range(notrees)]
        n, m = df.shape[0], random_features
        features = df.columns
        features_index = [i for i in range(len(features)) if features[i] not in ['ID', 'CLASS']]
        class_idx = list(np.where(features == 'CLASS')[0])
        for i in range(notrees):
            # We generate the random list of instances and features
            idx, fts = np.random.choice(n, size=n, replace=True),np.random.choice(features_index, size=m, replace=False)
            # We should not forget to include the CLASS column in addition to the features
            sub_df = df.iloc[idx, list(fts)+(class_idx)]
            class_freq = sub_df.CLASS.value_counts(normalize=True).sort_index()
            sub_features = features[fts]
            tree = self.divide_and_conquer(sub_df, sub_features, class_freq, min_samples_split, 0, i)
            self.models[i].append(tree)
            self.models[i] = sorted(self.models[i], key=lambda x:x[0])
                
                
    ## This recursive funtion takes a row and a nodeno and will go through the nodes to find the probabilities
    ## giving back a vector of size number of labels           
    def make_prediction(self, nodeno, row, notree):
        def closer_idx(value,keys):
            idx = np.argmin(np.abs(np.array(list((keys)), dtype='int')-int(value)))
            return(list(keys)[idx])
        nodeno, f, d = self.models[notree][int(nodeno)]
        if(f == "leaf"):
            return(d.sort_index())
        else:
            value = row[f]
            # We have some case where we don't have any node for the values of row
            if(value not in d.keys()):
                # if so we will go in the deepest node compatible and average the predictions of the sub_nodes
                v = list(d.values())
                m = np.zeros([len(v), len(self.labels)])
                for i in range(len(v)):
                     m[i] = self.make_prediction(v[i], row, notree)
                m = np.mean(m, axis = 0)
                return(m)
            else:
                # else we just iterate in the good next node
                new_nodeno = d[value]
            return(self.make_prediction(new_nodeno, row, notree))
    
        
        
    def predict(self, df):
        df = df.drop(['ID', 'CLASS'], axis = 1)
        df = apply_imputation(df_init=df, imputation=self.imputation)
        df = apply_bins(df_init=df, binning=self.binning)
        matrix = np.array([np.zeros(len(self.labels)) for i in range (df.shape[0])])
        for i in range(df.shape[0]):
            row = df.iloc[i]
            mat_pred_i = np.zeros([self.notrees, len(self.labels)])
            # We iterate the prediction over all trees and average the result
            for j in range(self.notrees):
                mat_pred_i[j] = self.make_prediction(0, row, j)
            mat_pred_avg = np.mean(mat_pred_i, axis = 0)
            matrix[i] = mat_pred_avg
        # We need to add the predictions of the labels that might not have been present in the training set
        # for example label 4 is never represented so it should appear wth a prob of 0 for all rows
        probs = pd.DataFrame(matrix, columns=self.labels)
        max_i = np.max(a=np.array(probs.columns, dtype='int'))
        for i in range(1,max_i):
            if (i not in probs.columns):
                probs[i] = 0
        cols = sorted(predictions.columns)
        probs = probs.loc[:,cols]
        return(probs)

In [750]:
glass_train_df = pd.read_csv("glass_train.txt")

glass_test_df = pd.read_csv("glass_test.txt")

forest_model = DecisionForest()

test_labels = glass_test_df["CLASS"]

min_samples_split_values = [1,2,5]
random_features_values = [1,2,5]

parameters = [(min_samples_split,random_features) for min_samples_split in min_samples_split_values 
              for random_features in random_features_values]

results = np.empty((len(parameters),3))

for i in range(len(parameters)):
    t0 = time.perf_counter()
    forest_model.fit(glass_train_df,min_samples_split=parameters[i][0],random_features=parameters[i][1])
    print("Training time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    t0 = time.perf_counter()
    predictions = forest_model.predict(glass_test_df)
    print("Testing time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    results[i] = [accuracy(predictions,test_labels),brier_score(predictions,test_labels),
                  auc(predictions,test_labels)] # Assuming that you have defined auc - remove otherwise

results = pd.DataFrame(results,index=pd.MultiIndex.from_product([min_samples_split_values,random_features_values]),
                       columns=["Accuracy","Brier score","AUC"])

results

Training time (1, 1): 0.48 s.
Testing time (1, 1): 0.49 s.
Training time (1, 2): 2.59 s.
Testing time (1, 2): 0.71 s.
Training time (1, 5): 18.16 s.
Testing time (1, 5): 1.05 s.
Training time (2, 1): 0.72 s.
Testing time (2, 1): 0.51 s.
Training time (2, 2): 2.20 s.
Testing time (2, 2): 0.92 s.
Training time (2, 5): 18.90 s.
Testing time (2, 5): 1.00 s.
Training time (5, 1): 0.81 s.
Testing time (5, 1): 0.86 s.
Training time (5, 2): 2.33 s.
Testing time (5, 2): 0.80 s.
Training time (5, 5): 8.71 s.
Testing time (5, 5): 0.82 s.


Accuracy  Brier score       AUC
1 1  0.579439     0.631169  0.746790
  2  0.626168     0.559199  0.771412
  5  0.616822     0.455764  0.869111
2 1  0.663551     0.582253  0.816432
  2  0.626168     0.508361  0.860150
  5  0.616822     0.453809  0.864737
5 1  0.654206     0.590970  0.799225
  2  0.682243     0.529303  0.852432
  5  0.663551     0.453548  0.882252

## Instructions results

In [751]:
forest_model = DecisionForest()
train_labels = glass_train_df["CLASS"]
forest_model.fit(glass_train_df,min_samples_split=1)
predictions = forest_model.predict(glass_train_df)
print("Accuracy on training set: {0:.2f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.2f}".format(auc(predictions,train_labels)))
print("Brier score on training set: {0:.2f}".format(brier_score(predictions,train_labels)))

Accuracy on training set: 0.85
AUC on training set: 0.96
Brier score on training set: 0.35


In [18]:
glass_train_df = pd.read_csv("glass_train.txt")

glass_test_df = pd.read_csv("glass_test.txt")

forest_model = DecisionForest()

test_labels = glass_test_df["CLASS"]

min_samples_split_values = [1,2,5]
random_features_values = [1,2,5]

parameters = [(min_samples_split,random_features) for min_samples_split in min_samples_split_values 
              for random_features in random_features_values]

results = np.empty((len(parameters),3))

for i in range(len(parameters)):
    t0 = time.perf_counter()
    forest_model.fit(glass_train_df,min_samples_split=parameters[i][0],random_features=parameters[i][1])
    print("Training time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    t0 = time.perf_counter()
    predictions = forest_model.predict(glass_test_df)
    print("Testing time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    results[i] = [accuracy(predictions,test_labels),brier_score(predictions,test_labels),
                  auc(predictions,test_labels)] # Assuming that you have defined auc - remove otherwise

results = pd.DataFrame(results,index=pd.MultiIndex.from_product([min_samples_split_values,random_features_values]),
                       columns=["Accuracy","Brier score","AUC"])

results

Training time (1, 1): 7.51 s.
Testing time (1, 1): 0.09 s.
Training time (1, 2): 10.65 s.
Testing time (1, 2): 0.09 s.
Training time (1, 5): 19.78 s.
Testing time (1, 5): 0.09 s.
Training time (2, 1): 7.97 s.
Testing time (2, 1): 0.09 s.
Training time (2, 2): 12.01 s.
Testing time (2, 2): 0.10 s.
Training time (2, 5): 19.42 s.
Testing time (2, 5): 0.10 s.
Training time (5, 1): 4.57 s.
Testing time (5, 1): 0.10 s.
Training time (5, 2): 7.31 s.
Testing time (5, 2): 0.09 s.
Training time (5, 5): 9.75 s.
Testing time (5, 5): 0.09 s.


Accuracy  Brier score       AUC
1 1  0.644860     0.477912  0.860736
  2  0.710280     0.411981  0.892842
  5  0.710280     0.421511  0.895716
2 1  0.663551     0.452421  0.872447
  2  0.626168     0.411684  0.911137
  5  0.654206     0.435847  0.881621
5 1  0.644860     0.457859  0.865636
  2  0.719626     0.410616  0.903016
  5  0.616822     0.455782  0.883897

In [24]:
train_labels = glass_train_df["CLASS"]
forest_model.fit(glass_train_df,min_samples_split=1)
predictions = forest_model.predict(glass_train_df)
print("Accuracy on training set: {0:.2f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.2f}".format(auc(predictions,train_labels)))
print("Brier score on training set: {0:.2f}".format(brier_score(predictions,train_labels)))

Accuracy on training set: 0.96
AUC on training set: 1.00
Brier score on training set: 0.12


### Comment on assumptions, things that do not work properly, etc.